# The power of constrained language models.

**Why and how to build constrained language models with a custom beam search algorithm. A guide with Hugging Face code.**

Pre-trained generative language models (such as OpenAI's GPT2 and GPT3) or seq2seq models (such as T5 or the recently released T0) generate free-flowing natural language. This means that their output sentences can have any shape. To get the most value out of these models, we would sometimes like the outputs to follow a certain structure. In this notebook I will show you how to achieve this and gain more value out of your language model using a custom beam search algorithm.

This notebook is ment to accompany a blogpost. Read the blogpost to fully understand the benefits of a custom beam search algorithm.

## Installing the necessary packages

In [ ]:
!pip install transformers
!pip install torch
!pip install numpy

     |████████████████████████████████| 2.9 MB 19.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.6 MB/s 
     |████████████████████████████████| 596 kB 39.0 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## The code







First we will download our model and corresponding tokenizer. I'm currently using GPT2 but any generative language model which has a `beam_search` method should work.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

We define a helper function: `set_scores_to_inf_for_banned_tokens`. Ignore this for now

In [ ]:
# src: https://huggingface.co/transformers/v4.1.1/_modules/transformers/generation_logits_process.html

def set_scores_to_inf_for_banned_tokens(scores, banned_tokens):
    """
    Modifies the scores in place by setting the banned token positions to `-inf`. Banned token is expected to be a
    list of list of banned tokens to ban in the format [[batch index, vocabulary position],...

    Args:
        scores: logits distribution of shape (batch size, vocabulary size)
        banned_tokens: list of list of tokens to ban of length (batch_size)
    """
    banned_mask_list = []
    for idx, batch_banned_tokens in enumerate(banned_tokens):
        for token in batch_banned_tokens:
            banned_mask_list.append([idx, token])
    if not banned_mask_list:
        return scores

    banned_mask = torch.LongTensor(banned_mask_list)
    indices = torch.ones(len(banned_mask))

    banned_mask = (
        torch.sparse.LongTensor(banned_mask.t(), indices, scores.size()).to(scores.device).to_dense().bool()
    )
    scores = scores.masked_fill(banned_mask, -float("inf"))
    return scores

We implement the `LogitsProcessor` class to get our desired effect. Our custom class should implement the `__call__` method of `LogitsProcessor`. 

This method will be called during each step of the beam search algorithm. The method takes as input the `input_ids` sequence of the partially generated beam and the `scores` of the next possible tokens.

By manipulating these `scores` based on the tokens present in the `input_ids`, we can control the structure of the generated sentence.

We implement two custom `LogitsProcessor` classes: `EvenLogits` and `ABCLogits`. The `EvenLogits` class makes sure that all generated tokens contain an even amount of characters. The `ABCLogits` class is a bit more complex: it makes sure that our generated tokens follow an 'a -> b -> c' pattern. The first token starts with an 'a', the second with a 'b', the third with a 'c', the fourth again with an 'a', etc.

In both implementations, we achieve this by dynamically creating a list of all tokens we are not allowed to output and then setting the corresponding `scores` to `-inf` using our helper function `set_scores_to_inf_for_banned_tokens`.

In [ ]:
from transformers import LogitsProcessor
import numpy as np

class EvenLogits(LogitsProcessor):
  def __call__(self, input_ids, scores):

    banned_tokens = []
    for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
      elementwise_length = np.vectorize(len)
      keys = np.array(list(tokenizer.vocab.keys()))
      values = np.array(list(tokenizer.vocab.values()))

      # indexes of tokens that are too long
      indexes = np.where(elementwise_length(keys) % 2 == 0)[0]

      banned_tokens.append(values[indexes])

    scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
    return scores

class ABCLogits(LogitsProcessor):
  def __init__(self, vocab):
    """
    vocab is a dictionary where the keys are tokens
    and the values are the corresponding ids.
    """
    # create an array of tokens
    # remove the 'Ġ' token (used to represent a blank space in the tokenizer)
    self.keys = list(tokenizer.vocab.keys())
    index_to_pop = self.keys.index('Ġ') 
    self.keys.pop(index_to_pop)
    self.keys = np.array(self.keys)

    # create an array of ids
    # also remove the 'Ġ' token
    self.values = list(tokenizer.vocab.values())
    self.values.pop(index_to_pop)
    self.values = np.array(self.values)

    # vectorized function used to get the first character of a token
    # ignores leading whitespaces and 'Ġ' tokens
    first_char = lambda x: x.strip('Ġ ')[0].lower()
    self.first_char = np.vectorize(first_char)

    # get the indexes of all IDs that do not start with the given letter
    not_a_indexes = np.where(self.first_char(self.keys) != 'a')
    not_b_indexes = np.where(self.first_char(self.keys) != 'b')
    not_c_indexes = np.where(self.first_char(self.keys) != 'c')

    # create sets of tokens that do not start with 'a', 'b' or 'c'
    self.not_a_values = self.values[not_a_indexes]
    self.not_b_values = self.values[not_b_indexes]
    self.not_c_values = self.values[not_c_indexes]

  def __call__(self, input_ids, scores):
    banned_tokens = []
    # for every beam (partially generated sentence)
    for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
      # get the last token of this beam
      last_word = tokenizer.decode(beam_input_ids[-1])
      # get the first character of this last token
      starting_char = self.first_char(last_word)
      # if the last token starts with 'a',
      # ban all words that do not start with 'b', etc.
      if starting_char == 'a':
        banned_tokens.append(self.not_b_values)
      elif starting_char == 'b':
        banned_tokens.append(self.not_c_values)
      elif starting_char == 'c':
        banned_tokens.append(self.not_a_values)
      else:
        banned_tokens.append(self.not_a_values)
    # set the scores of all banned tokens over the beams to -inf
    scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
    return scores

We use our custom `LogitsProcessor` classes during the beam search algorithm by passing them to the `logits_processor` attribute of the `beam_search` method of our model.

In the code blow, we use GPT2 to continue to prompt 'My cute dog is a' in the 'a -> b -> c' pattern.

In [ ]:
from transformers import (
    BeamSearchScorer,
    LogitsProcessorList,
    StoppingCriteriaList,
    MaxLengthCriteria
)
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 10
# how many beams to return after the algorithm
num_return_beams = 10

# the prompt to continue
prompt = 'My cute dog is a'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt' )
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size = prompt_tokenized.shape[0],
    num_beams = num_beams,
    num_beam_hyps_to_keep = num_return_beams,
    device=model.device
)

# instantiating a list of LogitsProcessor instances
# using our custom ABCLogits class
logits_processor = LogitsProcessorList([ABCLogits(tokenizer.vocab)])

# running beam search using our custom LogitsProcessor
generated = model.beam_search(
    torch.cat([prompt_tokenized] * num_beams),
    beam_scorer,
    logits_processor = logits_processor,
    stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=12)])
)

# printing the output beams
for index, output_tokenized in enumerate(generated):
  output = tokenizer.decode(output_tokenized)
  print(f'beam {index}: {output}')

beam 0: My cute dog is a bit confused about being called a bitch
beam 1: My cute dog is a bit confused about being called a baby
beam 2: My cute dog is a big cat and big cats are big
beam 3: My cute dog is a bit confused about being called a bunny
beam 4: My cute dog is a bit confused about being called a big
beam 5: My cute dog is a bit confused about being called a boy
beam 6: My cute dog is a bit confused about being called a b
beam 7: My cute dog is a bit confused about being called a bad
beam 8: My cute dog is a bit confused about being confused about being
beam 9: My cute dog is a bit confused about being called a black


Notice how the output of the model adheres to this structure without us having to provide the model with additional examples. I particularly like how the model still manages to make some coherent sentences despite these constraints.

Let's now add an additional constraint: all tokens should have an even amount of characters.

In [ ]:
# how many beams to track during the Viterbi algorithm
num_beams = 10
# how many beams to return after the algorithm
num_return_beams = 10

# the prompt to continue
prompt = 'My cute dog is a'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt' )
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size = prompt_tokenized.shape[0],
    num_beams = num_beams,
    num_beam_hyps_to_keep = num_return_beams,
    device=model.device
)

# instantiating a list of LogitsProcessor instances
# using our custom ABCLogits and EvenLogits class
# logits_processor = LogitsProcessorList([ABCLogits(tokenizer.vocab), EvenLogits()])
logits_processor = LogitsProcessorList([EvenLogits(), ABCLogits(tokenizer.vocab)])

# running beam search using our custom LogitsProcessor
generated = model.beam_search(
    torch.cat([prompt_tokenized] * num_beams),
    beam_scorer,
    logits_processor = logits_processor,
    stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=12)])
)

# printing the output beams
for index, output_tokenized in enumerate(generated):
  output = tokenizer.decode(output_tokenized)
  print(f'beam {index}: {output}')

beam 0: My cute dog is a better chance at becoming chaps by
beam 1: My cute dog is a better choice as both cute as baby
beam 2: My cute dog is a better chance at better care at best
beam 3: My cute dog is a better choice as both cute as before
beam 4: My cute dog is a better chance at building confidence as both
beam 5: My cute dog is a better chance at becoming champion at basketball
beam 6: My cute dog is a better chance at better conditions at better
beam 7: My cute dog is a better chance at better care at better
beam 8: My cute dog is a better chance at becoming champion at boxing
beam 9: My cute dog is a better chance at becoming chaps before


We again notice how the generated part of the sentence adheres to the ' a -> b -> c' constraint. Additionally, all generated words have an even amount of tokens.

These are quite heavy constraints, causing the model to give less coherent outputs.

## Conclusion

By writing our own version of the beam search algorithm, we are able to constrain the output of a pre-trained language model. This can be applied to generative models such as GPT2 and GPT3 and even seq2seq models such as T5 and T0. This is particularly useful when we want the output of our models to follow a certain pre-defined structure or adhere to a set of rules.

If you are interested in getting even more value out of your GPT model, check out [this](https://towardsdatascience.com/almost-no-data-and-no-time-unlocking-the-true-potential-of-gpt3-a-case-study-b4710ca0614a) post about how 'prompt engineering' can help you unlock more value out of your generative language models.